In [ ]:
!pip install kaggle

In [ ]:
# configuring the path of our kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# write api command to download the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# we got a sentiment140.zip file , so we need to extract it
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip :
  zip.extractall()
  print("The data is extracted :)")


The data is extracted :)


In [ ]:
# import the necessary dependencies
import numpy as np   # for array and matrices
import pandas as pd   # for making a dataframe which is easy to use
import re # stands for regular expression , used for pattern matching operations
from nltk.corpus import stopwords    # for removing all the stopwords
from nltk.stem.porter import PorterStemmer    # for stemming of left over words
from sklearn.feature_extraction.text import TfidfVectorizer    # for tokenizing of words
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# print all the stopwords in english
print(stopwords.words('english'))         # these are the words basically which do not have any sort of root word in them and so can be skipped in pre - processing

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Data Processing
tdata = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',  encoding='ISO-8859-1')

In [ ]:
tdata.shape

(1599999, 6)

In [ ]:
tdata.head(12)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [ ]:
# adding the column names and reading the data again
column_names = ['target','id','date','flag','user','text']
tdata = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names , encoding='ISO-8859-1')

In [ ]:
tdata.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
tdata.isnull().sum() # there are no missing value

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
tdata.describe()

,target,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [ ]:
# as there are positive , neutral as well as negative tweets , we'll check the distribution of that
tdata['target'].value_counts()


,count
target,
0,800000
4,800000


In [ ]:
# almost equal distribution , no neutral comment tho , 0 for negative , 4 for positive
tdata.replace({'target':{4:1}},inplace=True)

In [ ]:
tdata['target'].value_counts()

,count
target,
0,800000
1,800000


O means negative tweet , 1 means postive , there is no value for neutral tweet

**stemming** , the process of reducing a word to its root word by removing characters from the back ( suffix), for example : "swimmer," "swimming," and "swims" all have the root word "swim".

**lemmatization** , Analyzes the context of the sentence to find the root form, or "lemma". For example, the verb "running" would be identified as "run"

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content) :
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)     # removing every character except the alphabetical ones
  stemmed_content = stemmed_content.lower()    # converting all character to lowercase
  stemmed_content = stemmed_content.split()   # splitting the content for each tweet , and making a corresponding list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
tdata['stemmed_content'] = tdata['text'].apply(stemming)          # takes 50 min to complete this step

In [ ]:
tdata.head(5)    # As we added a new label for stemmed_content

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(tdata['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(tdata['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# separating the data and label //
X = tdata['stemmed_content'].values
Y = tdata['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
# splitting the data in to train and test data
x_train , x_test , y_train , y_test = train_test_split(X,Y,test_size=0.2,stratify = Y , random_state = 2)

# stratify = Y means we need an equal split of output label {0,1} in the test as well as train , otherwise all similar labels may go into test
# random_state = 2 , each time the data for me and other friend too will be splitted in the same manner

In [ ]:
print(X.shape,Y.shape,y_test.shape,x_test.shape,x_train.shape)

(1600000,) (1600000,) (320000,) (320000,) (1280000,)


In [ ]:
# we can't feed the textual data directly to the model , So we used a vectorizer to tokenize the text into numerical values , Here's how

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)       # we've to fit transform only once , after that only transform
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

  (0, 436713)	0.2725987626483838
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.1870933868497303
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.20289715703997938
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.44569393722995737
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277746
  (1279996, 434014)	0.2718945052332447
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.3521250099983204
  (1279996, 238077)

All Set ! We just need to train the Logistic Regression model

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Now we'll evaluate our model

x_train_prediction = model.predict(x_train)
train_accuracy = accuracy_score(y_train,x_train_prediction)
print("Accuracy for training data : " , train_accuracy)

Accuracy for training data :  0.79871953125


In [ ]:
x_test_prediction = model.predict(x_test)
test_accuracy = accuracy_score(y_test,x_test_prediction)
print("Accuracy for testing data : " , test_accuracy)

Accuracy for testing data :  0.77668125


In [ ]:
# If the the training and test data are having accuracy value close to each other that means , it has performed well
# if the train accuracy is much higher than the test accuracy , our model is overfitted and need to be resplitted more towards test

# we can use here Grid Search CV too for better hyperparameter tuning

Saving the trained model :

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
# Using the saved model for futurre predictions
loaded_model = pickle.load(open('trained_model.sav','rb'))

x_new = x_test[3] ;
print(y_test[3])

prediction = model.predict(x_new)
print(prediction)

if(prediction==0):
  print("negative tweet")
else :
  print("postive tweet :)")

0
[0]
negative tweet


In [ ]:
# prompt: I want to make two separate word clouds for positive and negative tweets using wordcloud or plotly

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming 'tdata' DataFrame and 'stemmed_content' column exist from the previous code

# Separate positive and negative tweets
positive_tweets = tdata[tdata['target'] == 1]['stemmed_content']
negative_tweets = tdata[tdata['target'] == 0]['stemmed_content']

# Create word clouds
def create_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title)
    plt.show()


create_wordcloud(positive_tweets, "Positive Tweets Word Cloud")
create_wordcloud(negative_tweets, "Negative Tweets Word Cloud")

In [ ]:
print("Please Bhagwaan Ji , placement ka dekhlena 20 LPA+ Achi company pls")

Please Bhagwaan Ji , placement ka dekhlena 20 LPA+ Achi company pls
